# JPEG
## Introduction
**J**oint **P**hotographic **E**xperts **G**roup ou **JPEG** est une norme qui définit le format d'enregistrement et l'algorithme de décodage pour une représentation numérique compressée d'une image fixe.
La compression **JPEG** repose sur la **suppression** de **données redondantes** et **non perceptibles** dans une image.

## Pourquoi est-ce intérécent de connaître comment ça fonctionnent?
- La plus part des images de votre téléphone ou de votre caméra son enregisté dans le format JPEG.
- environ 86% des images sur le web sont au format JPEG.
- La plus part des vidéos utilise le principe de ce format pour être compréssée.

Cette algorithme est partout dans notre vie et occupu une place importante dans celui-ci.

## Globalement qu'est ce que ça fait?
L'algorithme de compression va analyser chaque section d'une image, il va trouver et supprimer chaque pixels que nos yeux ne peuvent pas voir facilement.

## Pourquoi ça marche
Les oeils hummain ne sont pas parfait, ils ont leurs défaux. JPEG utilise ces défaux pour supprimer les informations que nos oeils on du mal à percevoir.
Un oeils est composé de Tiges qui nous permettent de capter la luminosité mais aussi, de Cônes pour capter les couleurs (Rouge, Vers, Bleu).
Dans chaque oeils nous avons cent million de Tiges et seulement 6 millions de Cônes.
Nos oeils sont beacoup plus réceptif à la luminosité et à l'obscurité plus tôt que aux couleurs.


## Comment ça marche ?
### Conversion de l'espace couleur
Une image est composé de pixels, ces pixels sont un mélange de rouge, vert et bleu allant de 0 à 255. Le mélange de ces couleur nous permet d'obtenir une couleur spécifique pour un pixel.

Le processus de **conversion de l'espace couleur** prend ces trois valeurs (Rouge, Vert, Bleu) pour chaque pixel, et calcul trois nouvelle valeurs (Luminosité, Chrominance Bleu, Chrominance Rouge). Ce processus est réversible, il est appelé **YCbCr**. Durant ce processus aucune données n'est perdu, c'est juste une nouvelle façon de représenter les couleurs.

In [1]:
# Ouvre le fichier image
from PIL import Image

# Ouvre le fichier image
img = Image.open("image.bmp")

# Transforme l'image en tableau de pixels RGB
pixels = img.load()

# Fonction qui transforme un tuple RGB en tuple YCbCr
def RGBtoYCbCr(RGB):
    R, G, B = RGB
    Y = 0.299 * R + 0.587 * G + 0.114 * B
    Cb = -0.168736 * R - 0.331264 * G + 0.5 * B + 128
    Cr = 0.5 * R - 0.418688 * G - 0.081312 * B + 128
    return (Y, Cb, Cr)

# Fonction qui transforme un tuple YCbCr en tuple RGB
def YCbCrtoRGB(YCbCr):
    Y, Cb, Cr = YCbCr
    R = Y + 1.402 * (Cr - 128)
    G = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
    B = Y + 1.772 * (Cb - 128)
    return (R, G, B)

# Affiche l'image
img.show()

# Transforme chaque pixel du tableau de de tuple RGB en tuple YCbCr
for i in range(img.size[0]):
    for j in range(img.size[1]):
        YCbCr = RGBtoYCbCr(pixels[i,j])
        pixels[i,j] = (int(YCbCr[0]), int(YCbCr[1]), int(YCbCr[2]))

# Affiche l'image aprés transformation
img.show()

Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.


### Sous-échantillonnage de la chrominance
Cette méthode permet par la suite de supprimer des données qui ne sont pas perceptible par nos yeux, comme nous l'avons dit plus tôt nos yeux sont movais pour detecter la couleur. Cette méthode est basé sur le fait que nos yeux sont beaucoup plus réceptif à la luminosité qu'aux couleurs.

Dans cette methode nous prenons les valeurs de chrominance bleu et rouge. Par la suite nous les divisons les deux images obtenus en blocks de pixels de 2 par 2. Puis nous calculons la moyenne de chaque block pour supprimer les données redondantes pour faire en sorte que chaque valeur moyenne d'un block de 4 pixels soit la même (en occupe 1 seul). En conséquence, les informations selon lesquelles nos yeux sont incapable de percevoir sont réduites de 75%, soit 1/4 de la taille d'origine, mais la luminosité reste intacte.

In [2]:
# Fonction pour diviser l'image en chrominance bleue et rouge
def extract_blue_chrominance(pixel):
    y, cb, cr = pixel
    return (0, cb, 0)

def extract_red_chrominance(pixel):
    y, cb, cr = pixel
    return (0, 0, cr)

def extract_luminance(pixel):
    y, cb, cr = pixel
    return (y, 0, 0)

# Fonction pour calculer la moyenne des blocs de 2x2
def average_block(pixels, color):
    width, height = img.size
    for i in range(0, width - 1, 2):
        for j in range(0, height - 1, 2):
            # Prendre les pixels dans un bloc de 2x2
            block = [
                pixels[i, j],
                pixels[i + 1, j] if i + 1 < width else pixels[i, j],
                pixels[i, j + 1] if j + 1 < height else pixels[i, j],
                pixels[i + 1, j + 1] if (i + 1 < width and j + 1 < height) else pixels[i, j]
            ]
            # Calculer la moyenne des valeurs de chrominance dans le bloc
            avg_cb = sum(p[1] for p in block) // len(block)
            avg_cr = sum(p[2] for p in block) // len(block)                    
            # Mettre à jour les images de chrominance bleue et rouge
            if color == "blue":
                for x in range(min(2, width - i)):
                    for y in range(min(2, height - j)):
                        blue_pixels[i + x, j + y] = (0, avg_cb, 0)
            elif color == "red":
                for x in range(min(2, width - i)):
                    for y in range(min(2, height - j)):
                        red_pixels[i + x, j + y] = (0, 0, avg_cr)

# Créez une nouvelle image pour la chrominance bleue
blue_chrominance = Image.new("YCbCr", img.size)
blue_pixels = blue_chrominance.load()

# Créez une nouvelle image pour la chrominance rouge
red_chrominance = Image.new("YCbCr", img.size)
red_pixels = red_chrominance.load()

# Créez une nouvelle image pour la luminance
luminance = Image.new("YCbCr", img.size)
luminance_pixels = luminance.load()

# Extraire la chrominance bleue, rouge et la luminance
for i in range(img.size[0]):
    for j in range(img.size[1]):
        blue_pixels[i, j] = extract_blue_chrominance(pixels[i, j])
        red_pixels[i, j] = extract_red_chrominance(pixels[i, j])
        luminance_pixels[i, j] = extract_luminance(pixels[i, j])

# Affichage des images de chrominance bleue et rouge
blue_chrominance.show()
red_chrominance.show()

# Moyenne des blocks de 2x2
average_block(blue_pixels, "blue")
average_block(red_pixels, "red")

# Affichage des images de chrominance bleue et rouge aprés moyenne des block de 2x2
blue_chrominance.show()
red_chrominance.show()

Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.
Ouverture dans une session de navigateur existante.


### Transformée en cosinus discrète et Quantification

Par la suite, ces deux étapes supprime également des informations, mais elle le font en exploitant le fait que nos yeux sont pas doués pour percevoir les éléments à hautre fréquence dans les images. Mais qu'est ce que ça veut dire ?

![edge](./edge.bmp)

Sur cette image de l'oreil droite du tigre, nos yeux sont capable de voir les contours de l'oreil, mais pas les détails à l'intérieur de l'oreil. C'est ce que l'on appelle les éléments à haute fréquence. Les éléments à haute fréquence sont des éléments qui changent rapidement dans une image. Les éléments à basse fréquence sont des éléments qui changent lentement dans une image.

La plus part des images des photographies de nature ou de paysage comportent des parties de l'image qui sont floues et la suppréssion des variations de couleurs à haute fraiquence pour créer des textures plus douces ne sont pas perceptible par nos yeux. Alors, comment l'algorithme JPEG exploite les nuance de l'oeil humain pour supprimer les éléments à haute fréquence ?

L'algorithme JPEG utilise une technique appelée **Transformée en cosinus discrète** ou **DCT**. Cette technique prend un block de 8 par 8 pixels et le transforme en un tableau de 8 par 8 valeurs. Chaque valeur représente la quantité de chaque fréquence dans le block. La valeur en haut à gauche représente la fréquence la plus basse et la valeur en bas à droite représente la fréquence la plus haute. Nous enlevons 128 à chaque valeur pour que les valeurs soient comprises entre -128 et 127. -128 représente la fréquence la plus basse et 127 la fréquence la plus haute.

![dct](./dct.png)

Dans cette illustration de la DCT, les zones sombres représentent les basses fréquences et les zones claires les hautes fréquences. L'information relative aux hautes fréquences, souvent négligeable pour la perception humaine, est donc regroupée vers les coins de la matrice, tandis que l'information des basses fréquences, plus cruciale pour la perception, est située près du coin supérieur gauche.
Prenons par exemple le deuxième carré en haut à gauche vers le bas. Plus la valeur de celui-ci sera haute, plus il y auras de dégradé de blanc vers le noir dans l'image. Si la valeur est basse, il y auras moins de dégradé de blanc vers le noir dans l'image. L'ensemble de ces valeurs nous permettent de reconstruire l'image.

Cette concentration des informations selon les capacités de perception humaine permet une réduction significative de la taille des données sans altérer sensiblement la qualité visuelle perçue.

Voici la formule de la DCT:

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/e06f6ee04c9c879a283edcbb7b1fc18b86fcec5b" alt="DCT"/>

u est la fréquence spatiale horizontale, pour les entiers 0 ≤ u < 8

v est la fréquence spatiale verticale, pour les entiers 0 ≤ v < 8

x et y sont les coordonnées horizontales et verticales du pixel, pour 0 ≤ x < 8 et 0 ≤ y < 8

Le processus s'applique sur la chrominance bleu et rouge, mais aussi sur la luminosité.

In [25]:
import math

luminance_minus_128 = []
blue_minus_128 = []
red_minus_128 = []

# Soustraction de 128 pour chaque pixel
for i in range(img.size[0]):
    luminance_minus_128.append([])
    blue_minus_128.append([])
    red_minus_128.append([])
    for j in range(img.size[1]):
        luminance_minus_128[i].append(luminance_pixels[i, j][0] - 128)
        blue_minus_128[i].append(blue_pixels[i, j][1] - 128)
        red_minus_128[i].append(red_pixels[i, j][2] - 128)

def alpha(u):
    return 1 / math.sqrt(2) if u == 0 else 1

def discrete_cosine_transform(matrix):
    width, height = img.size
    dct = []
    for u in range(0, width, 8):
        dct.append([])
        for v in range(0, height, 8):
            # 8x8 block. if the image isn't divisible by 8, we place 0 in the missing cells
            block = [[matrix[x][y] if x < len(matrix) and y < len(matrix[x]) else 0 for y in range(v, v + 8)] for x in range(u, u + 8)]
            # apply dct
            dct_block = [[0 for y in range(8)] for x in range(8)]
            for x in range(8):
                for y in range(8):
                    for i in range(8):
                        for j in range(8):
                            dct_block[x][y] += block[i][j] * math.cos(((2 * i + 1) * x * math.pi) / 16) * math.cos(((2 * j + 1) * y * math.pi) / 16)
                    dct_block[x][y] *= 1 / 4 * alpha(x) * alpha(y)
            dct[-1].append(dct_block)
    return dct
            

# DCT de la luminance
luminance_dct = discrete_cosine_transform(luminance_minus_128)
# DCT de la chrominance bleue
blue_dct = discrete_cosine_transform(blue_minus_128)
# DCT de la chrominance rouge
red_dct = discrete_cosine_transform(red_minus_128)

print("DCT de la luminance")
print(luminance_dct)

DCT de la luminance
[[[[1015.9999999999998, 8.038873388460928e-14, -1.1807095289301988e-13, 6.029155041345696e-14, 4.019436694230464e-14, 2.8136056859613244e-13, -2.009718347115232e-13, -1.2183917479386094e-13], [6.883285338869669e-13, -1.7763568394002505e-14, 1.0658141036401503e-14, 3.552713678800501e-15, 3.552713678800501e-15, 0.0, 8.881784197001252e-15, -6.217248937900877e-15], [8.541302975239736e-14, -1.0658141036401503e-14, 0.0, 0.0, 0.0, -3.552713678800501e-15, 0.0, 8.881784197001252e-16], [4.521866281009272e-14, 3.552713678800501e-15, -7.105427357601002e-15, 3.552713678800501e-15, 3.552713678800501e-15, 0.0, -5.329070518200751e-15, 4.440892098500626e-15], [1.004859173557616e-13, -7.105427357601002e-15, 0.0, -3.552713678800501e-15, 3.552713678800501e-15, 3.552713678800501e-15, -1.7763568394002505e-15, -2.6645352591003757e-15], [2.8136056859613244e-13, 2.842170943040401e-14, 0.0, 1.0658141036401503e-14, 1.0658141036401503e-14, 8.881784197001252e-15, 6.217248937900877e-15, 8.881784

A ce stade, nous avons la luminosité, la chrominance bleu ainsi que la chrominance rouge.
Nous avons aussi créé une variable qui représente les valeurs du dct sous forme matricielle.

Maintenant nous devons prendre des blocks de 64 pixels, soit 8x8 pixels pour chaques chrominances et pour la luminosité.



### Longueur d'exécution et codage de Huffman